<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150 align="right">
<img src=images/Acme.jpeg width=100 align="left">

#  Tokens

<img src="images/vault-token.png" width=300 >
 * Almost all operations in Vault requires a token.  It is important to understand the token lifecycle and various parameters.

## Several ways to control Token lifecycle:
* create a token with no MAX TTL (periodic token)
* create a token with no parent (orphan token)
* create a token with use limit

### Vault Setup

### Prerequisites
* Running Vault Enterprise instance (also in dev mode) 


In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [ ]:
vault login root

In [ ]:
vault status; vault version

---
### Create Short-Lived Tokens

In [ ]:
# in seconds
vault token create -ttl=120

**Using cURL**

In [ ]:
cat << EOF > /tmp/token.json
{
  "policies": ["web", "stage"],
  "meta": {
    "user": "armon"
  },
  "ttl": "10m",
  "renewable": true
}
EOF

curl --header "X-Vault-Token: root" --request POST \
  --data @/tmp/token.json \
$VAULT_ADDR/v1/auth/token/create  |jq


##### We'll use the value of token above in subsequent commands

In [ ]:
#vault token lookup <token_task_1>
vault token lookup s.FLIqAVNPLCKafRlndJLwY9Q7


### Token Renewal

In [ ]:
#vault token renew -increment=240 <token_task_2>
vault token renew -increment=100 s.KupeTj2SpKUKWzLTYdgpNbWa


In [ ]:
vault token lookup  s.1eBah74nejO3GSiJFhqyEXtJ

---
### Create a Token with Use Limit


In [ ]:
vault token create -use-limit=2


##### Look up information about the token consume 1 of the token's uses.

In [ ]:
VAULT_TOKEN=s.dDMmACkXERwE39VMC5dyIPWL vault token lookup


##### Write a key/value path cubby/hole/test to consume another of the token's uses.


In [ ]:
VAULT_TOKEN=s.h03vBSAtiWFCwRQrFalQbjEY  vault write cubbyhole/test name="student01"


##### Fail to read the key/value at path cubby/hole/test as the token is out of uses.

In [ ]:
VAULT_TOKEN=s.h03vBSAtiWFCwRQrFalQbjEY vault read cubbyhole/test

---
### Create an Orphan Token


In [ ]:
vault token create -ttl=90


##### Create a child token of the above token with a longer TTL of 180 seconds

In [ ]:
VAULT_TOKEN=s.XI0Q0cKxEHcv5oD41i2iLr4k vault token create -ttl=180


##### The remaining steps refer to it as *child_token*

After 90 seconds, the token expires!  This automatically revokes its child token.  If you try to look up the child token, you should receive **bad token** error.


In [ ]:
#use <child_token>
vault token lookup s.9FQkBGjkatVGgQ73W4aMX2jq

### Root or sudo users have the ability to generate **orphan** tokens.
Orphan tokens do not expire when their parents does.   Orphan tokens still expire when their own Max TTL is reached.

In [ ]:
vault token create -policy="" -orphan

In [ ]:
vault write auth/token/create-orphan policies=""

---
### Create a Token Role and Periodic Token

A common use case of periodic token is long-running processese where genration of a new token can interrupt the entire system flow.  The followings illustrate the creation of a role and periodic token for such long-running process.

In [ ]:
vault path-help auth/token


##### Create a token role named `monitor`.  This role has default policy and token renewal period of 24 hours.

In [ ]:
vault write auth/token/roles/monitor allowed_policies="default" period="24h"

In [ ]:
vault token create -role="monitor"

##### This token can be renewed multiple times indefinitely as long as it gets renewed before it expires.

---
### Enable Username and Password Auth Method

In [ ]:
vault auth list

\
&nbsp;
Userpass auth method allows users to login with username and password.

In [ ]:
vault auth enable userpass


##### Create a user with name `student01`, password `training` with `default` policies

In [ ]:
vault write auth/userpass/users/student01 password="training" policies="default"

##### Login with the user `student01` and their password `training`

In [ ]:
vault login -method=userpass username=student01 password="training"

---
### Generate batch tokens

Batch token explanation ....

In [ ]:
vault token create -type=batch -policy=default -ttl=360

##### Enable another `userpass` auth method at 'userpass-batch' which generates batch tokens.
<img src=images/think.png width=100>

In [ ]:
vault auth enable -path="userpass-batch" -token-type=batch userpass

##### Create a user called `john` with the password `training`

In [ ]:
vault write auth/userpass-batch/users/john password="training" policies="default"

In [ ]:
vault write transit/keys/orders/config min_decryption_version=2

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 class="center">